In [1]:
from pyspark.sql import SparkSession        

spark = (SparkSession
  .builder
  .appName("Flights")
  .getOrCreate())

21/12/24 12:51:22 WARN Utils: Your hostname, DESKTOP-4UGC9QT resolves to a loopback address: 127.0.1.1; using 172.27.101.137 instead (on interface eth0)
21/12/24 12:51:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/24 12:51:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/24 12:51:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark

In [5]:
flightsCSV = "./databricks-datasets/learning-spark-v2/flights/departuredelays.csv"

df = (spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load(flightsCSV))

In [8]:
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [9]:
df.count()

1391578

In [11]:
schema = "`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING"
df2 = spark.read.schema(schema).option("inferSchema", "true").option("header", "true").csv(flightsCSV)
df2.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [12]:
df2.show(10,False)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
|01030605|0    |602     |ABE   |ATL        |
|01041243|10   |602     |ABE   |ATL        |
|01040605|28   |602     |ABE   |ATL        |
|01051245|88   |602     |ABE   |ATL        |
|01050605|9    |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [15]:
df2.createOrReplaceTempView("flights")
spark.sql("select count(*) from flights").show()

+--------+
|count(1)|
+--------+
| 1391578|
+--------+



In [17]:
spark.sql("""SELECT delay, origin, destination, 
              CASE
                  WHEN delay > 360 THEN 'Very Long Delays'
                  WHEN delay >= 120 AND delay <= 360 THEN 'Long Delays'
                  WHEN delay >= 60 AND delay < 120 THEN 'Short Delays'
                  WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                  WHEN delay = 0 THEN 'No Delays'
                  ELSE 'Early'
               END AS Flight_Delays
               FROM flights
               ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows

